In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks')

In [0]:
import common_machine_learning as common
import pandas as pd
import numpy as np
import math

In [0]:
train = pd.read_csv("/content/drive/My Drive/Colab Notebooks/sets_de_datos/dima_train_with_features_2.csv")

In [0]:
X_test = pd.read_csv("/content/drive/My Drive/Colab Notebooks/sets_de_datos/dima_test_with_features_2.csv")

In [0]:
mega_set = train.drop(['precio'], axis = 1).append(X_test, ignore_index = True)

In [0]:
mega_set_comercial = mega_set.loc[mega_set['tipodepropiedad'].isin(['Terreno comercial', 
                                             'Local Comercial', 
                                             'Oficina comercial',
                                             'Local en centro comercial', 
                                             'Bodega comercial'])]

In [0]:
mega_set_comercial_lat_lng = mega_set_comercial[['tipodepropiedad', 'lat', 'lng']].copy()

In [0]:
mega_set_comercial_lat_lng['lat_lng'] = mega_set_comercial_lat_lng.apply(lambda x: (x['lat'], x['lng']), axis = 1)

In [0]:
lat_lng_todas_comerciales = mega_set_comercial_lat_lng.groupby(['tipodepropiedad']).agg({'lat_lng' : 'unique'}).to_dict()

In [40]:
lat_lng_todas_comerciales

{'lat_lng': {'Bodega comercial': array([(20.56931401022, -100.38684368134001),
         (20.61292242196524, -100.46709326017066),
         (25.66462147303325, -100.21518230438231), ..., (24.0212, -104.643),
         (20.610027, -100.385849), (19.395010300000006, -98.94559790000001)],
        dtype=object), 'Local Comercial': array([(19.5120829, -99.2614547),
         (20.70482447245631, -103.34198604580305),
         (19.4526689, -99.1383674), ...,
         (18.897666399999505, -96.943147516667),
         (22.121405192406005, -101.01945877075195),
         (24.102031797623038, -93.38817892986329)], dtype=object), 'Local en centro comercial': array([(19.45938175679808, -99.13342247287204),
         (20.8149134, -103.45421450000003),
         (20.6088697, -103.39643569999998),
         (25.69199280577214, -100.35689674451048),
         (32.51319069689765, -117.01793301942736),
         (19.65180422292952, -99.20628673541262),
         (19.41177052929376, -99.25308182188064),
         (20

In [0]:
def distancia_euclideana_minima(lat, lng, array_lat_lng):
  distancia_minima =\
    ((lat - array_lat_lng[0][0])**2 + (lng - array_lat_lng[0][1]))**(1/2)
  for tupla in array_lat_lng[1:]:
    try:
      aux = np.sqrt(np.square(lat - tupla[0]) + np.square(lng - tupla[1]))
      distancia_minima = min(distancia_minima, aux)
    except:
      print(lat, tupla[0], lng, tupla[1])
      raise TypeError("type error")
  return distancia_minima

In [0]:
for tipodepropiedad in ['Terreno comercial', 'Local Comercial', 'Oficina comercial',
                        'Local en centro comercial', 'Bodega comercial']:
      mega_set['distancia_euclideana_minima_a_' + "_".join(tipodepropiedad.split(' '))] =\
        mega_set.apply(lambda x: 
                       distancia_euclideana_minima(
                          x['lat'], 
                          x['lng'], 
                          lat_lng_todas_comerciales['lat_lng'][tipodepropiedad]),
                        axis = 1)

20.689350810546344 19.008372494200025 -103.44156611524352 -99.1946834741255


In [0]:
for tipodepropiedad in ['Terreno comercial', 'Local Comercial', 'Oficina comercial',
                        'Local en centro comercial', 'Bodega comercial']:
      train['distancia_euclideana_minima_a_' + "_".join(tipodepropiedad.split(' '))] =\
        train.apply(lambda x: 
                       distancia_euclideana_minima(
                          x['lat'], 
                          x['lng'], 
                          lat_lng_todas_comerciales['lat_lng'][tipodepropiedad]),
                        axis = 1)

In [0]:
for tipodepropiedad in ['Terreno comercial', 'Local Comercial', 'Oficina comercial',
                        'Local en centro comercial', 'Bodega comercial']:
      X_test['distancia_euclideana_minima_a_' + "_".join(tipodepropiedad.split(' '))] =\
        X_test.apply(lambda x: 
                       distancia_euclideana_minima(
                          x['lat'], 
                          x['lng'], 
                          lat_lng_todas_comerciales['lat_lng'][tipodepropiedad]),
                        axis = 1)

In [0]:
train.to_csv("/content/drive/My Drive/Colab Notebooks/sets_de_datos/dima_train_with_features_3.csv", index = False)

In [0]:
X_test.to_csv("/content/drive/My Drive/Colab Notebooks/sets_de_datos/dima_test_with_features_3.csv", index = False)